In [20]:
import mlflow
import pickle
import pandas as pd
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from datetime import datetime
from sklearn.metrics import root_mean_squared_error, mean_squared_error

In [3]:
# access to tracking server
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops-datatalk/02_experiment_tracking/mlruns/1', creation_time=1715068447646, experiment_id='1', last_update_time=1715068447646, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-datatalk/02_experiment_tracking/mlruns/0', creation_time=1715068447638, experiment_id='0', last_update_time=1715068447638, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
# create experiment
client.create_experiment(name = "my-cool-experiment")

'2'

In [6]:
# filter RMSE based on selected criteria
runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "metrics.rmse < 7",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 56a535cc40b342b68c60dc87c8f559be, rmse: 6.3184
run id: 43bf9d8c7219414ca6825be0b45da865, rmse: 6.3184
run id: 7b0bc226fb294f8481733b0eadfe2af7, rmse: 6.3191
run id: 3a14344d25f74b0b885474d69021fe26, rmse: 6.3339
run id: 6be1480a674c4038a8080c3bb4dad233, rmse: 6.3597


In [14]:
# register model
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "56a535cc40b342b68c60dc87c8f559be"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri = model_uri, name = "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1715073570179, current_stage='None', description=None, last_updated_timestamp=1715073570179, name='nyc-taxi-regressor', run_id='56a535cc40b342b68c60dc87c8f559be', run_link=None, source='/home/ubuntu/mlops-datatalk/02_experiment_tracking/mlruns/1/56a535cc40b342b68c60dc87c8f559be/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [15]:
# view all registered models
model_name = "nyc-taxi-regressor"
latest_versions = client.search_model_versions(f"name='{model_name}'")

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None
version: 2, stage: None
version: 1, stage: None


In [18]:
model_version = 3
new_stage = "Staging"

client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions = False
)

/tmp/ipykernel_16763/1882954017.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1715073570179, current_stage='Staging', description=None, last_updated_timestamp=1715073721544, name='nyc-taxi-regressor', run_id='56a535cc40b342b68c60dc87c8f559be', run_link=None, source='/home/ubuntu/mlops-datatalk/02_experiment_tracking/mlruns/1/56a535cc40b342b68c60dc87c8f559be/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [19]:
# update registered model data
date = datetime.today().date()

client.update_model_version(
    name = model_name,
    version = model_version,
    description = f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1715073570179, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-05-07', last_updated_timestamp=1715073900113, name='nyc-taxi-regressor', run_id='56a535cc40b342b68c60dc87c8f559be', run_link=None, source='/home/ubuntu/mlops-datatalk/02_experiment_tracking/mlruns/1/56a535cc40b342b68c60dc87c8f559be/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [27]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient = 'records')

    return dv.transform(train_dicts)

def test_model(name, stage, x_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(x_test)

    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [23]:
df = read_dataframe("../data/green_tripdata_2021-03.parquet")

client.download_artifacts(run_id = '56a535cc40b342b68c60dc87c8f559be', path = 'preprocessor', dst_path = '.')

/home/ubuntu/anaconda3/envs/mlops-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/ubuntu/mlops-datatalk/02_experiment_tracking/preprocessor'

In [24]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

x_test = preprocess(df, dv)
target = "duration"
y_test = df[target].values

In [28]:
# model prediction
%time test_model(name = model_name, stage = "Staging", x_test = x_test, y_test = y_test)

/home/ubuntu/anaconda3/envs/mlops-env/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 21.4 s, sys: 7.33 ms, total: 21.4 s
Wall time: 5.71 s


{'rmse': 6.2702965482607915}

In [29]:
# model transition to production
client.transition_model_version_stage(
    name = model_name,
    version = 3,
    stage = "Production",
    archive_existing_versions = True
)

/tmp/ipykernel_16763/2723081698.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1715073570179, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-05-07', last_updated_timestamp=1715074473274, name='nyc-taxi-regressor', run_id='56a535cc40b342b68c60dc87c8f559be', run_link=None, source='/home/ubuntu/mlops-datatalk/02_experiment_tracking/mlruns/1/56a535cc40b342b68c60dc87c8f559be/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>